# Projeto 1 - Ciência dos Dados

Nome: Arthur Cisotto Machado

Nome: Alessandra Yumi Carvalho Ogawa

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [456]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import emoji
from emoji import UNICODE_EMOJI

In [457]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Arthur\Dropbox\A - ARTHUR\2o Semestre\CDados\P1_CDados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [458]:
# https://docs.python.org/3/library/re.html#


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[”/-@\\n:;?\"\'().,]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    text_split = text_subbed.split()
    return ' '.join(text_split).lower()

def separa_emoji(tweet):
    '''
    Essa função separa os emojis juntos, além de transcreve-los
    para suas respectivas strings descritivas, optimizando um pouco o processamento.
    Nota-se que alguns emojis não usados não estão no Unicode da língua portuguesa,
    adotamos assim o de língua inglesa como alternativa de transcrição.
    '''
    modified=' '.join(emoji.get_emoji_regexp().split(tweet))
    modified=modified.split()
    for i,emoji1 in enumerate(modified):
        if emoji1 in UNICODE_EMOJI['pt']:
            modified[i]=UNICODE_EMOJI['pt'][emoji1].replace(':','')
        elif emoji1 in UNICODE_EMOJI['en']:
            modified[i]=UNICODE_EMOJI['en'][emoji1].replace(':','')
        else:
            continue
    modified=' '.join(modified)
        
    return modified

def limpeza_e_emoji(tweet):
    texto_filtrado = cleanup(tweet)
    sem_emoji = separa_emoji(texto_filtrado)
    return sem_emoji


In [459]:
filename = 'la casa de papel.xlsx'

In [460]:
train = pd.read_excel(filename)
train['Treinamento'] = train['Treinamento'].apply(limpeza_e_emoji)
train.head(5)


,Treinamento,"Classificação (relevante = 1, não relevante = 0)"
0,tô vendo la casa de papel mdssss o primeiro ep...,1.0
1,agora vou assistir lá casa de papel e dormir p...,0.0
2,dedo coçando pra assistir lá casa de papel mas...,0.0
3,la casa de papel me faz torcer pros bandidos,1.0
4,la casa de papel acabou comigo me encontro des...,1.0


In [461]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)


,Teste,"Classificação (relevante = 1, não relevante = 0)"
0,quero terminar de ver a nova parte de la casa ...,0
1,não existe outra série no mundo que mexa comig...,1
2,tá foda de desviar de todos os spoilers de la ...,1
3,@todoroki_jun2 né tô me sentindo em lá casa de...,0
4,la casa de papel é tão ruim que nem consegui t...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

O produto escolhido para análise é a série La Casa de Papel da Netflix e Vancouver Media. 

Classificamos como relevantes tweets que mostravam a opinião e sentimentos, sejam eles positivos ou negativos, sobre a série.
*Consideramos tweets relacionados a spoilers como relevantes pois eles também demonstram um forte sentimento que as pessoas possuem em relação à série.

Classificamos como não relevantes tweets que não se encaixaram na nossa classificação de relevância, tweets que falavam sobre tópicos pessoais ou tweets que falavam sobre algum personagem específico da série.


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [462]:
filtro_nao_relevante = train['Classificação (relevante = 1, não relevante = 0)']==0
filtro_relevante = train['Classificação (relevante = 1, não relevante = 0)']==1


In [463]:
relevantes_train = train[filtro_relevante]
nao_relevantes_train = train[filtro_nao_relevante]

In [464]:
relevantes_train

,Treinamento,"Classificação (relevante = 1, não relevante = 0)"
0,tô vendo la casa de papel mdssss o primeiro ep...,1.0
3,la casa de papel me faz torcer pros bandidos,1.0
4,la casa de papel acabou comigo me encontro des...,1.0
5,nossa mas que inferno esse final de lá casa de...,1.0
6,vey chorei em vis a via igual uma criança agor...,1.0
...,...,...
291,eu não chorei vendo lá casa de papel não né ro...,1.0
296,eu achei que nenhuma morte de la casa de papel...,1.0
297,caralho eu to chorando igual uma vagabunda com...,1.0
298,cara eu nunca chorei tanto quanto eu chorei ol...,1.0


In [465]:
nao_relevantes_train

,Treinamento,"Classificação (relevante = 1, não relevante = 0)"
1,agora vou assistir lá casa de papel e dormir p...,0.0
2,dedo coçando pra assistir lá casa de papel mas...,0.0
10,denver o maior de la casa de papel e ngm me co...,0.0
12,o rio de lá casa de papel é maior prova de que...,0.0
13,krl vi agora q to vendo a psrte de la casa de ...,0.0
...,...,...
290,lucas tem que parar de me dar spoiler de la ca...,0.0
292,faltou só ela pra maratonar la casa de papel cmg,0.0
293,odeio mt essa grávida de la casa de papel q tr...,0.0
294,flaviagasi se pá tem uns que acham que é la ca...,0.0


# Base de Teste

In [466]:
## Salvar tweets como strings para limpar 
relevantes_train_txt = ''
for tweet in relevantes_train['Treinamento']:
    relevantes_train_txt += " "
    relevantes_train_txt += str(tweet)

nao_relevantes_train_txt = ''
for tweet in nao_relevantes_train['Treinamento']:
    nao_relevantes_train_txt += " "
    nao_relevantes_train_txt += str(tweet)




# Tabelas de Frequências

In [467]:
todas_palavras_relevantes = relevantes_train_txt.split()
todas_palavras_irrelevantes = nao_relevantes_train_txt.split()

serie_relevante = pd.Series(todas_palavras_relevantes)
serie_irrelevante = pd.Series(todas_palavras_irrelevantes)


In [468]:
tabela_relevante = serie_relevante.value_counts()
tabela_irrelevante = serie_irrelevante.value_counts()
tabela_relevante

de        332
casa      202
papel     199
la        159
eu         64
         ... 
vis         1
raiva       1
ok          1
fraco       1
normal      1
Length: 703, dtype: int64

# Probabilidades

In [469]:
palavras = relevantes_train_txt + nao_relevantes_train_txt
todas_palavras = palavras.split()
serie_palavras = pd.Series(todas_palavras)
tabela_palavras = serie_palavras.value_counts(normalize=True)
palavras_sem_repeticao = set(todas_palavras_relevantes + todas_palavras_irrelevantes)

In [470]:
probR = len(serie_relevante)/len(serie_palavras)
probIR = len(serie_irrelevante)/len(serie_palavras)

In [471]:
def probDadoconj(palavra, prob_conj, lista_palavras_conj):
    if palavra in lista_palavras_conj:
        return prob_conj[palavra]
    else:
        return 0

#exemplo: print(probDadoconj("la", tabela_relevante, todas_palavras_relevantes))

In [472]:
def aplicando_laplace(prob_dado_conj, lista_palavras_conj):
    return (prob_dado_conj+1)/(len(lista_palavras_conj)+len(palavras_sem_repeticao))


In [473]:
def Classificacao(tweet):
    prob_relevante = 1
    lista_tweet = tweet.split()
    for palavra in lista_tweet:
        prob = probDadoconj(palavra, tabela_relevante, todas_palavras_relevantes)
        prob_laplace = aplicando_laplace(prob, todas_palavras_relevantes)
        prob_relevante *= prob_laplace
    probRtweet = prob_relevante*probR
    prob_irrelevante = 1
    for palavra in lista_tweet:
        prob = probDadoconj(palavra, tabela_irrelevante, todas_palavras_irrelevantes)
        prob_laplace = aplicando_laplace(prob, todas_palavras_irrelevantes)
        prob_irrelevante *= prob_laplace
    probIRtweet = prob_irrelevante*probIR
    if probRtweet < probIRtweet: 
        return 0
    else: 
        return 1


In [474]:
train['Classificacao_Naive_Bayes'] = train.Treinamento.apply(Classificacao)
train

,Treinamento,"Classificação (relevante = 1, não relevante = 0)",Classificacao_Naive_Bayes
0,tô vendo la casa de papel mdssss o primeiro ep...,1.0,1
1,agora vou assistir lá casa de papel e dormir p...,0.0,0
2,dedo coçando pra assistir lá casa de papel mas...,0.0,0
3,la casa de papel me faz torcer pros bandidos,1.0,1
4,la casa de papel acabou comigo me encontro des...,1.0,1
...,...,...,...
295,tô aqui assistindo la casa de papel mesmo depo...,0.0,0
296,eu achei que nenhuma morte de la casa de papel...,1.0,1
297,caralho eu to chorando igual uma vagabunda com...,1.0,1
298,cara eu nunca chorei tanto quanto eu chorei ol...,1.0,1


In [475]:
#Porcentagem de acerto de acerto no conjunto de Treino:

tp=train.loc[(train['Classificacao_Naive_Bayes']==1)&(train['Classificação (relevante = 1, não relevante = 0)']==1),:].shape[0]
tn=train.loc[(train['Classificacao_Naive_Bayes']==0)&(train['Classificação (relevante = 1, não relevante = 0)']==0),:].shape[0]
print('A acurácia do modelo foi de: ',100*(tp+tn)/train.shape[0],'%')

A acurácia do modelo foi de:  94.0 %


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [477]:
test['Teste'] = test['Teste'].apply(limpeza_e_emoji)
test['Classificacao_Naive_Bayes'] = test['Teste'].apply(Classificacao)
test

,Teste,"Classificação (relevante = 1, não relevante = 0)",Classificacao_Naive_Bayes
0,quero terminar de ver a nova parte de la casa ...,0,0
1,não existe outra série no mundo que mexa comig...,1,1
2,tá foda de desviar de todos os spoilers de la ...,1,1
3,todoroki_jun né tô me sentindo em lá casa de p...,0,1
4,la casa de papel é tão ruim que nem consegui t...,1,0
...,...,...,...
195,em respeito ao meu casal fav de la casa de pap...,0,1
196,assistindo la casa de papel para poder falar m...,0,1
197,sexo não kk veste a roupa aí que a gente vai m...,0,0
198,meu deus o final de la casa de papel…………,1,1


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**